In [2]:
setwd('/data/import/project1//momo-6.12.0.199/runtime/PIPELINE/taxonkit3//GC')
# library(xlsx)
# con = read.xlsx("./ss.xlsx",sheetIndex = 1,header =T)
# con = con[,1:84]
# EGC = read.xlsx("./ss.xlsx",sheetIndex = 2,header =T)
# EGC = EGC[,1:84]
# GC = read.xlsx("./ss.xlsx",sheetIndex = 3,header =T)
# GC = GC[,1:84]
# library(tableone) 
# library(dplyr)
# data = rbind(con,EGC,GC) 
# data = data[!duplicated(data$patient.ID),2:84]

In [3]:
library(dplyr)

Warning message:
“程辑包‘dplyr’是用R版本4.2.3 来建造的”

载入程辑包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [4]:
data = readRDS('./raw.data.rds') %>%  filter(Group != 'Control')

In [5]:
table(data$Group)


EGC  GC 
509 527 

In [6]:
head(data,2)

,Group,Age,A.G,ADA,AFU,ALB,ALP,ALT,ALT.AST,AMY,⋯,CA50,CA724,Size,Sex,area,degree.of.differentiation,physiology,Lymph.Node.Metastasis,Depth.of.Invasion,Stage
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,EGC,60,1.44,10.8,NA,43.0,79,22,0.85,42,⋯,NA,NA,0.04,Female,A,G1G2,AC,No,T1,T1N0Mx
2,EGC,78,1.30,13.4,NA,43.3,87,16,0.94,29,⋯,NA,NA,3.60,Female,B,G2,AC,No,T1,T1N0M0


In [7]:
table(data$Stage)


     No  T1N0M0  T1N0Mx  T1N1M0  T2N0M0  T2N1M0  T3N0M0  T3N1M0  T3N2M0  T3N3M0 
    261     182      12       6      31       7      41      40       9       4 
 T3N3M1  T4N0M0  T4N1M0  T4N1M1  T4N1Mx  T4N2M0  T4N2M1  T4N2Mx  T4N3M0  T4N3M1 
      1      61     104       8       3     134      13       1      20      17 
TisN0M0 
     44 

In [8]:
library(tableone)

In [9]:
data$Sex  = ifelse(data$Sex %in% c('Male ','Male'),'Male','Female')

data[,c(2:76)] =  sapply(data[,c(2:76)], as.numeric)
data$Lymph.Node.Metastasis = gsub('是','Yes',data$Lymph.Node.Metastasis)
data$degree.of.differentiation = gsub('G2G1','G1G2',data$degree.of.differentiation)
data$Stage = gsub('?','',data$Stage)
data$area = gsub('胃','G',data$area)
data$area = gsub('-',NA,data$area)
data$degree.of.differentiation = gsub('-',NA,data$degree.of.differentiation)
data$physiology = gsub('-',NA,data$physiology)
data$Lymph.Node.Metastasis = gsub('-',NA,data$Lymph.Node.Metastasis)
data$Depth.of.Invasion = gsub('-',NA,data$Depth.of.Invasion)
data$Stage = gsub('-',NA,data$Stage)
data$Stage = gsub('No',NA,data$Stage)

data$Stage = gsub('T1','',data$Stage)
data$Stage = gsub('T2','',data$Stage)
data$Stage = gsub('T3','',data$Stage)
data$Stage = gsub('T4','',data$Stage)
data$Stage = gsub('Tis','',data$Stage)
#data[is.na(data)] <- "unknown"
# ##################################################

myVars = colnames(data)
data1 = data[,myVars]

delete.na <- function(DF, n=0) {
  DF[rowSums(is.na(DF)) <= n,]
}
data1 = delete.na(data1, 80)

data1$Group = as.factor(data1$Group)
data1$Sex = as.factor(data1$Sex)
data1$area = as.factor(data1$area)
data1$degree.of.differentiation = as.factor(data1$degree.of.differentiation)
data1$physiology = as.factor(data1$physiology)
data1$Lymph.Node.Metastasis = as.factor(data1$Lymph.Node.Metastasis)
data1$Depth.of.Invasion = as.factor(data1$Depth.of.Invasion)
data1$Stage = as.factor(data1$Stage)
catVars <- c("Sex")
data1$area = gsub('/','_',data1$area)
data1$N_Stage = substr(data1$Stage,1,2)
data1$M_Stage = substr(data1$Stage,3,4)

tab3  = CreateTableOne(vars = myVars, strata = "Group" , data = data1, factorVars = catVars,
               addOverall  = TRUE)
tab3Mat <- print(tab3, quote = FALSE, noSpaces = TRUE, printToggle = FALSE)
write.csv(tab3Mat, file = "myTable.csv")


In [22]:
table(data1$Stage)


N0M0 N0Mx N1M0 N1M1 N1Mx N2M0 N2M1 N2Mx N3M0 N3M1 
 359   12  157    8    3  143   13    1   24   18 

In [23]:
table(data1$M_Stage)


 M0  M1  Mx 
683  39  16 

In [12]:
data1 = data1 %>% select(-Stage)

In [114]:
saveRDS(data1,file = 'co.data.rds')

In [13]:
head(data1,2)

,Group,Age,A.G,ADA,AFU,ALB,ALP,ALT,ALT.AST,AMY,...,CA724,Size,Sex,area,degree.of.differentiation,physiology,Lymph.Node.Metastasis,Depth.of.Invasion,N_Stage,M_Stage
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>
1,EGC,60,1.44,10.8,NA,43.0,79,22,0.85,42,...,NA,0.04,Female,A,G1G2,AC,No,T1,N0,Mx
2,EGC,78,1.30,13.4,NA,43.3,87,16,0.94,29,...,NA,3.60,Female,B,G2,AC,No,T1,N0,M0


In [14]:
dim(data1)

[1] 1036   84

In [15]:
write.csv(data1,file = 'co.data1.csv',row.names =F)

In [118]:
library(dplyr)
# data1 = data1 %>% tibble::rownames_to_column(.,'id')
# data1$id = paste0('T',data1$id)
delete.na <- function(DF, n=0) {
  DF[rowSums(is.na(DF)) <= n,]
}
data1 = delete.na(data1, 80)


library(pROC)
require(xgboost)
require(Matrix)
require(data.table)
if (!require('vcd')) {
  install.packages('vcd')
}
library(caret)   #
library(openxlsx) #

In [119]:
df_all = data1

In [140]:
head(data1,2)

,Group,Age,A.G,ADA,AFU,ALB,ALP,ALT,ALT.AST,AMY,...,CA724,Size,Sex,area,degree.of.differentiation,physiology,Lymph.Node.Metastasis,Depth.of.Invasion,N_Stage,M_Stage
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<fct>,<chr>,<fct>,<fct>,<fct>,<fct>,<chr>,<chr>
1,EGC,60,1.44,10.8,NA,43.0,79,22,0.85,42,...,NA,0.04,Female,A,G1G2,AC,No,T1,N0,Mx
2,EGC,78,1.30,13.4,NA,43.3,87,16,0.94,29,...,NA,3.60,Female,B,G2,AC,No,T1,N0,M0


In [120]:

ohe_feats = c( "Sex", "area" ,"degree.of.differentiation",
               "physiology","Lymph.Node.Metastasis",
               "Depth.of.Invasion","N_Stage",'M_Stage')

dummies <- dummyVars(~ Sex +  area + degree.of.differentiation +
                       physiology +Lymph.Node.Metastasis +Depth.of.Invasion +N_Stage +  M_Stage
                     , data = df_all,fullRank=T)

In [ ]:

df_all_ohe <- as.data.frame(predict(dummies, newdata = df_all))

df_all_combined <- cbind(df_all[,-c(which(colnames(df_all) %in% ohe_feats))], df_all_ohe)

In [123]:
dim(df_all_combined)

[1] 1036  112

In [124]:
tail(df_all_combined,3)

,Group,Age,A.G,ADA,AFU,ALB,ALP,ALT,ALT.AST,AMY,...,physiology.mMA,Lymph.Node.Metastasis.Yes,Depth.of.Invasion.T2,Depth.of.Invasion.T3,Depth.of.Invasion.T4,N_StageN1,N_StageN2,N_StageN3,M_StageM1,M_StageMx
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1034,GC,59,1.39,14.7,NA,32.5,54,13,0.76,35,...,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
1035,GC,62,1.16,13.5,NA,40.6,83,16,0.76,60,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1036,GC,45,1.49,20.5,NA,40.1,58,53,1.04,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [125]:
library(tidyverse)
set.seed(1123)
library(missForest)
dat2 <- missForest(df_all_combined,ntree = 100)
head(dat2$ximp)
dat2$OOBerror

Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
"The response has five or fewer unique values.  Are you sure you want to do regression?"
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
"The response has five or fewer unique values.  Are you sure you want to do regression?"
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
"The response has five or fewer unique values.  Are you sure you want to do regression?"
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
"The response has five or fewer unique values.  Are you sure you want to do regression?"
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
"The response has five or fewer unique values.  Are you sure you want to do regression?"
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
"The response h

,Group,Age,A.G,ADA,AFU,ALB,ALP,ALT,ALT.AST,AMY,...,physiology.mMA,Lymph.Node.Metastasis.Yes,Depth.of.Invasion.T2,Depth.of.Invasion.T3,Depth.of.Invasion.T4,N_StageN1,N_StageN2,N_StageN3,M_StageM1,M_StageMx
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,EGC,60,1.4400,10.800,21.010,43.000,79.00,22.00,0.8500,42.0000,...,0,0,0.00,0.00,0,0,0,0,0,1
2,EGC,78,1.3000,13.400,19.540,43.300,87.00,16.00,0.9400,29.0000,...,0,0,0.00,0.00,0,0,0,0,0,0
3,EGC,66,25.7000,10.700,18.018,35.500,46.00,31.00,1.1900,60.3569,...,0,0,0.00,0.00,0,0,0,0,0,0
4,EGC,65,1.5977,12.254,18.447,42.799,122.23,27.61,1.1935,57.2000,...,0,0,0.00,0.00,0,0,0,0,0,0
5,EGC,58,1.8400,6.500,17.720,44.400,54.00,25.00,1.5600,81.0000,...,0,0,0.29,0.06,0,0,0,0,0,0
6,EGC,53,1.7400,10.000,14.539,43.900,52.00,151.00,2.2200,45.0000,...,0,0,1.00,0.00,0,0,0,0,0,0


NRMSE       PFC 
0.4636485 0.0000000

In [126]:
###  iris.narf < - randomForest(Species ~ ., iris.na, na.action=na.roughfix)
df_all_combined2 = dat2$ximp

In [127]:
head(df_all_combined2,2)

,Group,Age,A.G,ADA,AFU,ALB,ALP,ALT,ALT.AST,AMY,...,physiology.mMA,Lymph.Node.Metastasis.Yes,Depth.of.Invasion.T2,Depth.of.Invasion.T3,Depth.of.Invasion.T4,N_StageN1,N_StageN2,N_StageN3,M_StageM1,M_StageMx
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,EGC,60,1.44,10.8,21.01,43.0,79,22,0.85,42,...,0,0,0,0,0,0,0,0,0,1
2,EGC,78,1.30,13.4,19.54,43.3,87,16,0.94,29,...,0,0,0,0,0,0,0,0,0,0


In [128]:
#df_all_combined2 = df_all_combined2 %>%  filter(Group != 'GC')

In [134]:
dim(df_all_combined2)

[1] 1036  112

In [130]:
df_all_combined2$Group = ifelse(df_all_combined2$Group  == 'EGC',1,0)

In [131]:
##################Boruta  
set.seed(123)
trainIndex <- caret::createDataPartition(
  df_all_combined2$Group, 
  p = 0.6, 
  list = FALSE, 
  times = 1)

trainData <- df_all_combined2[trainIndex, ]
X_train <- trainData[, -1]
y_train <- trainData[, 1]

testData <- df_all_combined2[-trainIndex, ]
X_test <- testData[, -1]
y_test <- testData[, 1]

In [135]:
sel = readRDS('./sel.rds')

In [137]:
X_train = X_train[,sel]
X_test = X_test[,sel]

In [138]:
save(X_train,y_train,X_test,y_test,file = 'co.train-test.rda')